## Практическое задание к уроку 8. Рекуррентные нейронные сети RNN LSTM GRU
Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конкурирующими. Выяснить какая архитектура больше подходит для нашей задачи

построить свёрточные архитектуры

построить различные архитектуры с RNN

построить совместные архитектуры CNN -> RNN или (RNN -> CNN)


In [1]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=285dd2827c02ce9cf37f0bd267c84f6542fc4796db822d5ed48f2bf6df148eb2
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 8.2 MB 9.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=1cb2f33347cd3de32565ddca8156eb540955786ac9e33639fc90a4afb7906e2c
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [3]:
! pip install --upgrade xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 4.8 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [74]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import re

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Activation, Masking, Bidirectional, SimpleRNN, Dropout, LSTM, GRU, Flatten, MaxPooling1D
# from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
df =  pd.read_excel("отзывы за лето.xls")
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)
    
df['Content'] = df['Content'].apply(preprocess_text)
df.head()

,Rating,Content,Date
0,5,it just works,2017-08-14
1,4,целое удобноной приложениеиз минус хотеть боль...,2017-08-14
2,5,отлично,2017-08-14
3,5,зависать 1 работа антивирус ранее пользоваться...,2017-08-14
4,5,удобно работать быстро,2017-08-14


In [7]:
X_train, X_test, train_y, test_y = train_test_split(df.Content, df.Rating, test_size=0.3, random_state=42)

In [8]:
text_corpus_train = X_train.values
text_corpus_test = X_test.values

tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

x_train = pad_sequences(sequences_train, maxlen=training_length)
x_test = pad_sequences(sequences_test, maxlen=training_length)

In [9]:
print(word_count, training_length)

10580 113


In [10]:
num_classes = len(df.Rating.value_counts()) + 1

In [11]:
y_train = pd.DataFrame(train_y)
y_test = pd.DataFrame(test_y)

In [12]:
y_train = tf.keras.utils.to_categorical(y_train['Rating'], num_classes)
y_test = tf.keras.utils.to_categorical(y_test['Rating'], num_classes)

## CNN

In [16]:
model_cnn = Sequential()
model_cnn.add(Embedding(input_dim=word_count, output_dim=300, input_length=training_length))
model_cnn.add(Conv1D(128, 5))
model_cnn.add(Activation("relu"))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(100))
model_cnn.add(Activation("relu"))
model_cnn.add(Dense(num_classes))
model_cnn.add(Activation('softmax'))

In [17]:
model_cnn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 113, 300)          3174000   
                                                                 
 conv1d_1 (Conv1D)           (None, 109, 128)          192128    
                                                                 
 activation_3 (Activation)   (None, 109, 128)          0         
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)               12900     
                                                                 
 activation_4 (Activation)   (None, 100)               0         
                                                      

In [18]:
batch_size = 512
epochs = 5

history = model_cnn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/5
26/26 [==============================] - 1s 38ms/step - loss: 1.1370 - accuracy: 0.6828 - val_loss: 0.8955 - val_accuracy: 0.7132
Epoch 2/5
26/26 [==============================] - 1s 27ms/step - loss: 0.7499 - accuracy: 0.7552 - val_loss: 0.6987 - val_accuracy: 0.7498
Epoch 3/5
26/26 [==============================] - 1s 28ms/step - loss: 0.6039 - accuracy: 0.7906 - val_loss: 0.6686 - val_accuracy: 0.7664
Epoch 4/5
26/26 [==============================] - 1s 29ms/step - loss: 0.5186 - accuracy: 0.8183 - val_loss: 0.6699 - val_accuracy: 0.7643
Epoch 5/5
26/26 [==============================] - 1s 28ms/step - loss: 0.4353 - accuracy: 0.8577 - val_loss: 0.6854 - val_accuracy: 0.7685


In [19]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report


class Metrics(Callback):

    def __init__(self, model):
        self.model = model

    def on_test_end(self, epoch, logs=None):
        val_predict = self.model.predict(x_test).argmax(axis=1)
        print('\n', classification_report(test_y, val_predict))

In [20]:
model_cnn.evaluate(x_test, y_test, callbacks=[Metrics(model_cnn)])

194/194 [==============================] - ETA: 0s - loss: 0.6823 - accuracy: 0.7725
               precision    recall  f1-score   support

           1       0.54      0.69      0.61       697
           2       0.00      0.00      0.00       219
           3       0.26      0.16      0.20       283
           4       0.28      0.15      0.20       607
           5       0.87      0.95      0.91      4392

    accuracy                           0.77      6198
   macro avg       0.39      0.39      0.38      6198
weighted avg       0.72      0.77      0.74      6198

194/194 [==============================] - 1s 7ms/step - loss: 0.6823 - accuracy: 0.7725


[0.6823274493217468, 0.7725072503089905]

## RNN

In [47]:
model_rnn = Sequential()

model_rnn.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=300,
              trainable=True,
              mask_zero=True))
model_rnn.add(Masking(mask_value=0.0))

model_rnn.add(SimpleRNN(128, recurrent_dropout=0.2))
model_rnn.add(Dense(100, activation='relu'))
model_rnn.add(Dropout(0.5))
model_rnn.add(Dense(num_classes, activation='softmax'))

model_rnn.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

model_rnn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 113, 300)          3174000   
                                                                 
 masking_8 (Masking)         (None, 113, 300)          0         
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 128)               54912     
                                                                 
 dense_20 (Dense)            (None, 100)               12900     
                                                                 
 dropout_8 (Dropout)         (None, 100)               0         
                                                                 
 dense_21 (Dense)            (None, 6)                 606       
                                                                 
Total params: 3,242,418
Trainable params: 3,242,418
N

In [48]:
history = model_rnn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/5
26/26 [==============================] - 8s 257ms/step - loss: 0.4063 - accuracy: 0.6038 - val_loss: 0.2694 - val_accuracy: 0.7049
Epoch 2/5
26/26 [==============================] - 5s 205ms/step - loss: 0.2692 - accuracy: 0.7049 - val_loss: 0.2266 - val_accuracy: 0.7160
Epoch 3/5
26/26 [==============================] - 5s 204ms/step - loss: 0.2268 - accuracy: 0.7382 - val_loss: 0.2045 - val_accuracy: 0.7471
Epoch 4/5
26/26 [==============================] - 5s 206ms/step - loss: 0.2038 - accuracy: 0.7621 - val_loss: 0.2004 - val_accuracy: 0.7519
Epoch 5/5
26/26 [==============================] - 5s 203ms/step - loss: 0.1861 - accuracy: 0.7784 - val_loss: 0.2041 - val_accuracy: 0.7581


In [49]:
model_rnn.evaluate(x_test, y_test, callbacks=[Metrics(model_rnn)])

193/194 [============================>.] - ETA: 0s - loss: 0.2055 - accuracy: 0.7662
               precision    recall  f1-score   support

           1       0.45      0.72      0.56       697
           2       0.00      0.00      0.00       219
           3       0.00      0.00      0.00       283
           4       0.29      0.03      0.06       607
           5       0.84      0.96      0.90      4392

    accuracy                           0.77      6198
   macro avg       0.32      0.34      0.30      6198
weighted avg       0.67      0.77      0.70      6198

194/194 [==============================] - 5s 28ms/step - loss: 0.2055 - accuracy: 0.7659


[0.20546936988830566, 0.7658922076225281]

## LSTM

In [67]:
model_lstm = Sequential()

model_lstm.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=300,
              trainable=True,
              mask_zero=True))
model_lstm.add(Masking(mask_value=0.0))

model_lstm.add(LSTM(128, return_sequences=True, recurrent_dropout=0.2))
model_lstm.add(LSTM(64, recurrent_dropout=0.2))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(num_classes, activation='softmax'))

model_lstm.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

model_lstm.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 113, 300)          3174000   
                                                                 
 masking_15 (Masking)        (None, 113, 300)          0         
                                                                 
 lstm_9 (LSTM)               (None, 113, 128)          219648    
                                                                 
 lstm_10 (LSTM)              (None, 64)                49408     
                                                                 
 dense_34 (Dense)            (None, 64)                4160      
                                                                 
 dropout_15 (Dropout)        (None, 64)                0         
                                                                 
 dense_35 (Dense)            (None, 6)               

In [68]:
history = model_lstm.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/5
26/26 [==============================] - 33s 1s/step - loss: 0.4197 - accuracy: 0.6576 - val_loss: 0.2754 - val_accuracy: 0.7049
Epoch 2/5
26/26 [==============================] - 28s 1s/step - loss: 0.3055 - accuracy: 0.7000 - val_loss: 0.2592 - val_accuracy: 0.7049
Epoch 3/5
26/26 [==============================] - 27s 1s/step - loss: 0.2638 - accuracy: 0.7091 - val_loss: 0.2143 - val_accuracy: 0.7374
Epoch 4/5
26/26 [==============================] - 27s 1s/step - loss: 0.2241 - accuracy: 0.7387 - val_loss: 0.1996 - val_accuracy: 0.7436
Epoch 5/5
26/26 [==============================] - 26s 1s/step - loss: 0.2015 - accuracy: 0.7547 - val_loss: 0.1966 - val_accuracy: 0.7519


In [69]:
model_lstm.evaluate(x_test, y_test, callbacks=[Metrics(model_lstm)])

194/194 [==============================] - ETA: 0s - loss: 0.1937 - accuracy: 0.7620
               precision    recall  f1-score   support

           1       0.42      0.74      0.54       697
           2       0.00      0.00      0.00       219
           3       0.00      0.00      0.00       283
           4       0.00      0.00      0.00       607
           5       0.85      0.96      0.90      4392

    accuracy                           0.76      6198
   macro avg       0.25      0.34      0.29      6198
weighted avg       0.65      0.76      0.70      6198

194/194 [==============================] - 29s 150ms/step - loss: 0.1937 - accuracy: 0.7620


[0.1936846524477005, 0.7620199918746948]

## GRU

In [70]:
model_gru = Sequential()

model_gru.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=300,
              trainable=True,
              mask_zero=True))
model_gru.add(Masking(mask_value=0.0))

model_gru.add(GRU(128, return_sequences=True, recurrent_dropout=0.2))
model_gru.add(GRU(64, recurrent_dropout=0.2))
model_gru.add(Dense(64, activation='relu'))
model_gru.add(Dropout(0.5))
model_gru.add(Dense(num_classes, activation='softmax'))

model_gru.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

model_gru.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 113, 300)          3174000   
                                                                 
 masking_16 (Masking)        (None, 113, 300)          0         
                                                                 
 gru (GRU)                   (None, 113, 128)          165120    
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dense_36 (Dense)            (None, 64)                4160      
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                                 
 dense_37 (Dense)            (None, 6)               

In [71]:
history = model_gru.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/5
26/26 [==============================] - 27s 866ms/step - loss: 0.4167 - accuracy: 0.6234 - val_loss: 0.2439 - val_accuracy: 0.7049
Epoch 2/5
26/26 [==============================] - 25s 970ms/step - loss: 0.2620 - accuracy: 0.7023 - val_loss: 0.2177 - val_accuracy: 0.7139
Epoch 3/5
26/26 [==============================] - 24s 916ms/step - loss: 0.2217 - accuracy: 0.7444 - val_loss: 0.2064 - val_accuracy: 0.7360
Epoch 4/5
26/26 [==============================] - 23s 869ms/step - loss: 0.1930 - accuracy: 0.7789 - val_loss: 0.2014 - val_accuracy: 0.7478
Epoch 5/5
26/26 [==============================] - 23s 875ms/step - loss: 0.1733 - accuracy: 0.7975 - val_loss: 0.2029 - val_accuracy: 0.7498


In [72]:
model_gru.evaluate(x_test, y_test, callbacks=[Metrics(model_gru)])

194/194 [==============================] - ETA: 0s - loss: 0.2028 - accuracy: 0.7569
               precision    recall  f1-score   support

           1       0.45      0.66      0.54       697
           2       0.00      0.00      0.00       219
           3       0.00      0.00      0.00       283
           4       0.26      0.14      0.18       607
           5       0.85      0.94      0.90      4392

    accuracy                           0.76      6198
   macro avg       0.31      0.35      0.32      6198
weighted avg       0.68      0.76      0.71      6198

194/194 [==============================] - 26s 134ms/step - loss: 0.2028 - accuracy: 0.7569


[0.20283880829811096, 0.7568570375442505]

## Bi-LSTM

In [75]:
model_blstm = Sequential()

model_blstm.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=300,
              trainable=True,
              mask_zero=True))
model_blstm.add(Masking(mask_value=0.0))

# model_lstm.add(LSTM(128, return_sequences=True, recurrent_dropout=0.2))
model_blstm.add(Bidirectional(LSTM(128, recurrent_dropout=0.2)))
model_blstm.add(Dense(100, activation='relu'))
model_blstm.add(Dropout(0.5))
model_blstm.add(Dense(num_classes, activation='softmax'))

model_blstm.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

model_blstm.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 113, 300)          3174000   
                                                                 
 masking_18 (Masking)        (None, 113, 300)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 256)              439296    
 l)                                                              
                                                                 
 dense_38 (Dense)            (None, 100)               25700     
                                                                 
 dropout_17 (Dropout)        (None, 100)               0         
                                                                 
 dense_39 (Dense)            (None, 6)                 606       
                                                     

In [76]:
history = model_blstm.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/5
26/26 [==============================] - 37s 1s/step - loss: 0.3765 - accuracy: 0.6543 - val_loss: 0.2372 - val_accuracy: 0.7049
Epoch 2/5
26/26 [==============================] - 27s 1s/step - loss: 0.2415 - accuracy: 0.7283 - val_loss: 0.2078 - val_accuracy: 0.7374
Epoch 3/5
26/26 [==============================] - 27s 1s/step - loss: 0.2106 - accuracy: 0.7548 - val_loss: 0.1984 - val_accuracy: 0.7450
Epoch 4/5
26/26 [==============================] - 26s 1s/step - loss: 0.1907 - accuracy: 0.7747 - val_loss: 0.1957 - val_accuracy: 0.7478
Epoch 5/5
26/26 [==============================] - 26s 1s/step - loss: 0.1742 - accuracy: 0.7890 - val_loss: 0.1986 - val_accuracy: 0.7554


In [77]:
model_blstm.evaluate(x_test, y_test, callbacks=[Metrics(model_blstm)])

194/194 [==============================] - ETA: 0s - loss: 0.1967 - accuracy: 0.7627
               precision    recall  f1-score   support

           1       0.45      0.72      0.55       697
           2       0.00      0.00      0.00       219
           3       0.00      0.00      0.00       283
           4       0.21      0.04      0.07       607
           5       0.85      0.96      0.90      4392

    accuracy                           0.76      6198
   macro avg       0.30      0.34      0.30      6198
weighted avg       0.67      0.76      0.71      6198

194/194 [==============================] - 27s 140ms/step - loss: 0.1967 - accuracy: 0.7627


[0.19666892290115356, 0.7626653909683228]

## CNN + LSTM

In [90]:
model_cnnlstm = Sequential()
model_cnnlstm.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=300,
              trainable=True,
              mask_zero=True))
# model_cnnlstm.add(Conv1D(64, 3, padding='same', activation='relu'))
# model_cnnlstm.add(MaxPooling1D(2, padding='same'))
model_cnnlstm.add(Conv1D(128, 3, padding='same', activation='relu'))
model_cnnlstm.add(MaxPooling1D(2, padding='same'))


model_cnnlstm.add(LSTM(128, recurrent_dropout = 0.2))
# model_cnnlstm.add(LSTM(32))
model_cnnlstm.add(Dense(100, activation='relu'))
model_cnnlstm.add(Dropout(0.5))
model_cnnlstm.add(Dense(num_classes, activation='softmax'))

model_cnnlstm.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

model_cnnlstm.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, 113, 300)          3174000   
                                                                 
 conv1d_9 (Conv1D)           (None, 113, 128)          115328    
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 57, 128)          0         
 1D)                                                             
                                                                 
 lstm_17 (LSTM)              (None, 128)               131584    
                                                                 
 dense_48 (Dense)            (None, 100)               12900     
                                                                 
 dropout_22 (Dropout)        (None, 100)               0         
                                                     

In [91]:
history = model_cnnlstm.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/5
26/26 [==============================] - 9s 257ms/step - loss: 0.4085 - accuracy: 0.6532 - val_loss: 0.2429 - val_accuracy: 0.7049
Epoch 2/5
26/26 [==============================] - 6s 246ms/step - loss: 0.2483 - accuracy: 0.7280 - val_loss: 0.2045 - val_accuracy: 0.7402
Epoch 3/5
26/26 [==============================] - 6s 248ms/step - loss: 0.2070 - accuracy: 0.7649 - val_loss: 0.1967 - val_accuracy: 0.7457
Epoch 4/5
26/26 [==============================] - 6s 241ms/step - loss: 0.1866 - accuracy: 0.7807 - val_loss: 0.1981 - val_accuracy: 0.7554
Epoch 5/5
26/26 [==============================] - 6s 233ms/step - loss: 0.1688 - accuracy: 0.8041 - val_loss: 0.2021 - val_accuracy: 0.7519


In [92]:
model_cnnlstm.evaluate(x_test, y_test, callbacks=[Metrics(model_cnnlstm)])

194/194 [==============================] - ETA: 0s - loss: 0.2010 - accuracy: 0.7573
               precision    recall  f1-score   support

           1       0.48      0.67      0.56       697
           2       0.00      0.00      0.00       219
           3       0.00      0.00      0.00       283
           4       0.23      0.19      0.21       607
           5       0.87      0.94      0.90      4392

    accuracy                           0.76      6198
   macro avg       0.32      0.36      0.33      6198
weighted avg       0.69      0.76      0.72      6198

194/194 [==============================] - 7s 38ms/step - loss: 0.2010 - accuracy: 0.7573


[0.20096702873706818, 0.7573410868644714]

## LSTM+CNN

In [104]:
model_lstmcnn = Sequential()

model_lstmcnn.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=300))
model_lstmcnn.add(LSTM(128, return_sequences=True))
model_lstmcnn.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding="same"))
model_lstmcnn.add(MaxPooling1D(2))
model_lstmcnn.add(Flatten())
model_lstmcnn.add(Dense(100, activation='relu'))
model_lstmcnn.add(Dropout(0.5))
model_lstmcnn.add(Dense(num_classes, activation='softmax'))

model_lstmcnn.summary()
model_lstmcnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_29 (Embedding)    (None, 113, 300)          3174000   
                                                                 
 lstm_21 (LSTM)              (None, 113, 128)          219648    
                                                                 
 conv1d_13 (Conv1D)          (None, 113, 128)          49280     
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 56, 128)          0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 7168)              0         
                                                                 
 dense_56 (Dense)            (None, 100)               716900    
                                                     

In [105]:
history = model_lstmcnn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/5
26/26 [==============================] - 3s 65ms/step - loss: 0.3607 - accuracy: 0.6572 - val_loss: 0.2551 - val_accuracy: 0.7049
Epoch 2/5
26/26 [==============================] - 1s 47ms/step - loss: 0.2646 - accuracy: 0.7235 - val_loss: 0.2140 - val_accuracy: 0.7408
Epoch 3/5
26/26 [==============================] - 1s 47ms/step - loss: 0.2213 - accuracy: 0.7505 - val_loss: 0.1988 - val_accuracy: 0.7512
Epoch 4/5
26/26 [==============================] - 1s 47ms/step - loss: 0.1955 - accuracy: 0.7759 - val_loss: 0.1939 - val_accuracy: 0.7519
Epoch 5/5
26/26 [==============================] - 1s 47ms/step - loss: 0.1763 - accuracy: 0.7918 - val_loss: 0.1994 - val_accuracy: 0.7519


In [106]:
model_lstmcnn.evaluate(x_test, y_test, callbacks=[Metrics(model_lstmcnn)])

185/194 [===========================>..] - ETA: 0s - loss: 0.2010 - accuracy: 0.7606
               precision    recall  f1-score   support

           1       0.46      0.71      0.55       697
           2       0.00      0.00      0.00       219
           3       0.00      0.00      0.00       283
           4       0.24      0.11      0.15       607
           5       0.86      0.95      0.90      4392

    accuracy                           0.76      6198
   macro avg       0.31      0.35      0.32      6198
weighted avg       0.69      0.76      0.72      6198

194/194 [==============================] - 2s 11ms/step - loss: 0.1996 - accuracy: 0.7627


[0.19964781403541565, 0.7626653909683228]

**Вывод**

С нашей задачей лучше всего справляется CNN. Он дает accuracy выше 77% и отчет по классификации у него лучше. Все остальные сетки, в том числе комбинации RNN и CNN дают accuracy около 76% и ниже, при этом часто вообще не предсказывая 3 класса из 5